In [35]:
import pandas as pd

In [36]:
df = pd.read_csv('Sleep-health.csv')
df.head()

,Person.ID,Gender,Age,Occupation,Sleep.Duration,Quality.of.Sleep,Physical.Activity.Level,Stress.Level,BMI.Category,Systolic,Diastolic,Heart.Rate,Daily.Steps,Sleep.Disorder,BP.Category,AgeGroup
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126,83,77,4200,None,High Blood Pressure (Hypertension) Stage 1,20-30
1,2,Male,28,Doctor,6.2,6,60,8,Normal Weight,125,80,75,10000,None,High Blood Pressure (Hypertension) Stage 1,20-30
2,3,Male,28,Doctor,6.2,6,60,8,Normal Weight,125,80,75,10000,None,High Blood Pressure (Hypertension) Stage 1,20-30
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140,90,85,3000,Sleep Apnea,High Blood Pressure (Hypertension) Stage 2,20-30
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140,90,85,3000,Sleep Apnea,High Blood Pressure (Hypertension) Stage 2,20-30


In [37]:
cat_cols = list(df.select_dtypes(include=['object', 'category']).columns)
cat_cols = cat_cols[:2] + cat_cols[4:6]
cat_cols.append('BMI.Category')
cat_cols

['Gender', 'Occupation', 'BP.Category', 'AgeGroup', 'BMI.Category']

In [46]:
num_cols = list(df.select_dtypes(include=['int', 'float']).columns)

num_cols

['Person.ID',
 'Age',
 'Sleep.Duration',
 'Quality.of.Sleep',
 'Physical.Activity.Level',
 'Stress.Level',
 'Systolic',
 'Diastolic',
 'Heart.Rate',
 'Daily.Steps']

In [38]:
df.describe().drop(columns=['Person.ID'], axis=1)

,Age,Sleep.Duration,Quality.of.Sleep,Physical.Activity.Level,Stress.Level,Systolic,Diastolic,Heart.Rate,Daily.Steps
count,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000
mean,42.184492,7.132086,7.312834,59.171123,5.385027,128.553476,84.649733,70.165775,6816.844920
std,8.673133,0.795657,1.196956,20.830804,1.774526,7.748118,6.161611,4.135676,1617.915679
min,27.000000,5.800000,4.000000,30.000000,3.000000,115.000000,75.000000,65.000000,3000.000000
25%,35.250000,6.400000,6.000000,45.000000,4.000000,125.000000,80.000000,68.000000,5600.000000
50%,43.000000,7.200000,7.000000,60.000000,5.000000,130.000000,85.000000,70.000000,7000.000000
75%,50.000000,7.800000,8.000000,75.000000,7.000000,135.000000,90.000000,72.000000,8000.000000
max,59.000000,8.500000,9.000000,90.000000,8.000000,142.000000,95.000000,86.000000,10000.000000


In [39]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

In [45]:
X = df.drop(columns=['Sleep.Disorder'], axis=1)
y = df['Sleep.Disorder']

response_encoder = LabelEncoder()
encoder = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')
encoder.fit(X[cat_cols])

OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.25,
                                                    random_state = 42
                                                    )


X_train_encoded = encoder.transform(X_train[cat_cols])
X_test_encoded = encoder.transform(X_test[cat_cols])


train_labels = response_encoder.fit_transform(y_train)
test_labels = response_encoder.fit_transform(y_test)

In [48]:
num_cols = ['Age', 'Sleep.Duration','Quality.of.Sleep', 'Physical.Activity.Level', 'Stress.Level', 'Systolic', 'Diastolic', 'Heart.Rate', 'Daily.Steps']

In [49]:
import numpy as np
X_train_final = np.hstack([X_train_encoded, X_train[num_cols].values])
X_test_final = np.hstack([X_test_encoded, X_test[num_cols].values])

In [57]:
rfc = RandomForestClassifier(n_estimators=50)
rfc.fit(X_train_final, train_labels)

accuracy = rfc.score(X_test_final, test_labels)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.8936170212765957


In [58]:
from sklearn.metrics import classification_report

y_pred = rfc.predict(X_test_final)
print(classification_report(y_pred, test_labels))

              precision    recall  f1-score   support

           0       0.82      0.78      0.80        18
           1       0.98      0.93      0.96        58
           2       0.73      0.89      0.80        18

    accuracy                           0.89        94
   macro avg       0.84      0.87      0.85        94
weighted avg       0.90      0.89      0.90        94

